In [8]:
from datetime import datetime, timezone

import pandas as pd
from delta.tables import DeltaTable
from pyspark.sql.types import StringType
from pyspark.sql.functions import (col,
                                   lit,
                                   max as _max,
                                   concat_ws, 
                                   sha2)



StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 10, Finished, Available)

In [9]:
# create source table
data = {
    'name_id': [1,2],
    'name':['Pedro', 'John'],
    'dob' :['1980-01-25', '1990-04-12'],
    'town': ['Adelaide', 'Melbourne'],
    'updated_at': ['2024-03-25 12:10:03.534945', '2024-03-25 12:10:03.534945']
    }

source = pd.DataFrame(data)

df = spark.createDataFrame(source)

df.write.format('delta').mode('overwrite').saveAsTable('names')

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 11, Finished, Available)

In [10]:
# create sink table empty
(DeltaTable.createIfNotExists(spark)
    .tableName("names_scd2")
    .addColumn("hash_key", "STRING")
    .addColumn("name_id", "INT")
    .addColumn("name", "STRING")
    .addColumn("dob", "DATE")
    .addColumn("town", "STRING")
    .addColumn("from_date", "TIMESTAMP")
    .addColumn("to_date", "TIMESTAMP")
    .addColumn("current", "BOOLEAN")
    .addColumn("updated_at", "TIMESTAMP")
    .execute())

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 12, Finished, Available)

In [11]:
def create_hashkey(df):

    columns = [col for col in df.columns]
    
    # Convert non-string columns to strings
    for col in df.columns:
        df = df.withColumn(f's_{col}', df[col].cast(StringType()))

    # Concatenate all columns
    df = df.withColumn("concatenated_values", concat_ws("", *df.columns))

    # Compute SHA256 hash
    df = df.withColumn("hash_key", sha2(df["concatenated_values"], 256))

    columns.append("hash_key")

    df = df.select(columns)

    return df

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 13, Finished, Available)

In [12]:
def merge_scd2(updates_df, sink_table):
    changed_records = (updates_df.alias('updates')
             .join(sink_table.toDF().alias('scd2'), 'name_id')
             .where("""
                 `scd2`.`current` = true
                 and
                 `updates`.`hash_key` <> `scd2`.`hash_key`
             """)
            )

    print('changed records')
    display(changed_records)

    # stage updates
        # new records that will be inserted:  to_date will be nulls and from_date will be the last_updated
        # changed redords that will be updated and turned to_date to the last_updated 

    staged_updates = (changed_records
                    .selectExpr("null as merge_key", "updates.*") # changed updated records
                    .union(updates_df.selectExpr("name_id as merge_key", "*")) # new records either new version or brand new 
                    )


    print('staged_updates')
    display(staged_updates)

    # merge into scd2

    now = datetime.now(timezone.utc).isoformat()

    staged_updates = staged_updates.withColumn('updated_at', lit(now))

    # Apply SCD Type 2 operation using merge
    (sink_table.alias("scd2").merge(
    staged_updates.alias("staged_updates"),
    "scd2.name_id = merge_key")
    .whenMatchedUpdate(
    condition = """
                    `scd2`.`current` = true
                    and
                    `staged_updates`.`hash_key` <> `scd2`.`hash_key`
                """,
    set = {                # Set current to false and endDate to source's effective date.
        "current": "false",
        "to_date": "staged_updates.updated_at"
    }
    ).whenNotMatchedInsert(
    values = {
        "hash_key":"staged_updates.hash_key",
        "name_id": "staged_updates.name_id",
        "name": "staged_updates.name",
        "dob": "staged_updates.dob",
        "town": "staged_updates.town",
        "current": "true", # Set current to true along with other changes
        "from_date": "staged_updates.updated_at", 
        "to_date": "null"
        ,"updated_at": "staged_updates.updated_at"
    }
    ).execute()
    )


StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 14, Finished, Available)

In [13]:
# load source and sink table
source_table = spark.read.table('names')
sink_table = DeltaTable.forName(spark, "names_scd2")


StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 15, Finished, Available)

In [14]:
#get latest changes since last water mark

watermark = '2000-01-01'

source_table = spark.read.table('names')

updates_df = (source_table
                .filter(col('updated_at')>watermark)
                )

watermark = (source_table
                 .select(['updated_at'])
                 .agg(_max(col('updated_at')))   
                    ).collect()[0][0]

updates_df = create_hashkey(updates_df)
display(updates_df)

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, bbd31bf5-0bc8-43b2-b649-2f88a69552b3)

In [15]:
# merge updated into scd2 table
merge_scd2(updates_df, sink_table)

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 17, Finished, Available)

changed records


SynapseWidget(Synapse.DataFrame, ca030ff7-0dbd-4be6-ad1b-26440c5b61e1)

staged_updates


SynapseWidget(Synapse.DataFrame, d549409f-50ae-4cb0-92cd-5f3e7949cbf2)

In [37]:
# display sink table
sink_table = DeltaTable.forName(spark, "names_scd2")
display(sink_table.toDF())

StatementMeta(, d20fd09a-3ecf-4607-8f94-6d00d7adf875, 39, Finished, Available)

SynapseWidget(Synapse.DataFrame, a28d4565-a2f1-4f9c-b957-d278cf410aa4)

In [16]:
# load current state of source table
source_table = DeltaTable.forName(spark, "names")
display(source_table.toDF())

# change one records

source_table.update(
  condition = col("name_id") == 1,
  set = { "town": lit("Sao Paulo"),
          "updated_at": lit(datetime.now(timezone.utc).isoformat())   }
)

# load current state of source table
source_table = DeltaTable.forName(spark, "names")
display(source_table.toDF())

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1696ad4d-3f8e-46aa-b7b1-58823ae2d207)

SynapseWidget(Synapse.DataFrame, c706d16a-98d2-491b-a714-abd8149fb2ea)

In [17]:
%%sql

-- add 2 new records

insert into names (name_id, name, dob, town, updated_at)
values (3, 'Richard', '2001-01-04', 'Montreal', CURRENT_TIMESTAMP()),
       (4, 'Maria', '1980-10-04', 'New York', CURRENT_TIMESTAMP())

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 19, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [18]:
#get latest changes since last water mark

source_table = spark.read.table('names')

updates_df = (source_table
                .filter(col('updated_at')>watermark)
                )

watermark = (source_table
                 .select(['updated_at'])
                 .agg(_max(col('updated_at')))   
                    ).collect()[0][0]

updates_df = create_hashkey(updates_df)
display(updates_df)

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, b3cee198-2ee8-4f01-ae28-958848dabb59)

In [19]:
# merge updated into scd2 table
merge_scd2(updates_df, sink_table)

StatementMeta(, 839d3e92-a18c-420b-a3ac-596064da79fc, 21, Finished, Available)

changed records


SynapseWidget(Synapse.DataFrame, 63fccc8a-a2f6-4f66-bc3a-d4d1228057b9)

staged_updates


SynapseWidget(Synapse.DataFrame, 19980984-5778-4d38-b13b-0897439fa1b2)

In [42]:
# display sink table
sink_table = DeltaTable.forName(spark, "names_scd2")
display(sink_table.toDF())

StatementMeta(, d20fd09a-3ecf-4607-8f94-6d00d7adf875, 44, Finished, Available)

SynapseWidget(Synapse.DataFrame, df856328-7412-4793-bd9e-792e2c616d06)